In [10]:
import numpy
import torch
import gurobipy
from measure import Expectation
import copy_ as deepcopy

from statistics_ import rand_int,check_random_state
from exception import SampleSizeError,DistributionError
from collections import abc
from numbers import Number
import time
import math

In [11]:
# example of using gurobipy library
def testGRB():
    import gurobipy as gp
    from gurobipy import GRB

    # Create a new model
    model = gp.Model("example")

    # Add variables
    x = model.addVar(vtype=GRB.CONTINUOUS, name="x")
    y = model.addVar(vtype=GRB.CONTINUOUS, name="y")

    # Set the objective function
    model.setObjective(2*x + 3*y, GRB.MAXIMIZE)

    # Add constraints
    model.addConstr(x + 2*y <= 4, "c0")
    model.addConstr(3*x + y <= 5, "c1")

    print('Optimize the model')
    model.optimize()

    print('Print the results')
    for v in model.getVars():
        print(f'{v.varName}: {v.x}')

    print(f'Objective Value: {model.objVal}')


testGRB()

Optimize the model
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x734c5483
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 5e+00]
Presolve time: 0.03s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+30   2.500000e+30   5.000000e+00      0s
       2    6.6000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.06 seconds (0.00 work units)
Optimal objective  6.600000000e+00
Print the results
x: 1.2
y: 1.4
Objective Value: 6.6


In [12]:
class StochasticModel(object):
    # Stochastic model class
    def __init__(self, name=""):
        self._model = gurobipy.Model(env=gurobipy.Env(), name=name)
        # each and every instance must have state variables, local copy variables
        self.states = []
        self.local_copies = []
        # (discretized) uncertainties
        # stage-wise independent discrete uncertainties
        self.uncertainty_rhs = {}
        self.uncertainty_coef = {}
        self.uncertainty_obj = {}
        # indices of stage-dependent uncertainties
        self.uncertainty_rhs_dependent = {}
        self.uncertainty_coef_dependent = {}
        self.uncertainty_obj_dependent = {}
        # true uncertainties
        # stage-wise independent true continuous uncertainties
        self.uncertainty_rhs_continuous = {}
        self.uncertainty_coef_continuous = {}
        self.uncertainty_obj_continuous = {}
        self.uncertainty_mix_continuous = {}
        # stage-wise independent true discrete uncertainties
        self.uncertainty_rhs_discrete = {}
        self.uncertainty_coef_discrete = {}
        self.uncertainty_obj_discrete = {}
        # cutting planes approximation of recourse variable alpha
        self.alpha = None
        self.cuts = []
        # linking constraints
        self.link_constrs = []
        # number of discrete uncertainties
        self.n_samples = 1
        # number of state varibles
        self.n_states = 0
        # probability measure for discrete uncertainties
        self.probability = None
        # type of true problem: continuous/discrete
        self._type = None
        # collection of all specified dim indices of Markovian uncertainties
        self.Markovian_dim_index = []
        # risk measure
        self.measure = Expectation

    def __getattr__(self, name):
        try:
            return getattr(self._model, name)
        except AttributeError:
            raise AttributeError("no attribute named {}".format(name))
        
    # representation of true problem
    def __repr__(self):
        uncertainty = ""
        if self._type == "discrete":
            uncertainty_rhs = ("" if self.uncertainty_rhs == {} else "discrete uncertainties on the RHS of constraints, ")
            uncertainty_coef = ("" if self.uncertainty_coef == {} else "discrete uncertainties on the coefficients of constraints, ")
            uncertainty_obj = ("" if self.uncertainty_obj == {} else "discrete uncertainties in the objective, ")
            uncertainty = uncertainty_rhs + uncertainty_coef + uncertainty_obj
        elif self._type == "continuous":
            uncertainty_rhs_continuous = ("" if self.uncertainty_rhs_continuous == {} else "continuous uncertainties on the RHS of constraints, ")
            uncertainty_coef_continuous = ("" if self.uncertainty_coef_continuous == {} else "continuous uncertainties on the coefficients of constraints, ")
            uncertainty_obj_continuous = ("" if self.uncertainty_obj_continuous == {} else "continuous uncertainties in the objective, ")
            uncertainty_mix_continuous = (""if self.uncertainty_mix_continuous == {} else "continuous uncertainties in multiple locations, ")
            uncertainty = (uncertainty_rhs_continuous + uncertainty_coef_continuous + uncertainty_obj_continuous + uncertainty_mix_continuous)
        comma = "" if uncertainty == "" else ", "
        return ("<Stochastic " + repr(self._model)[1:-1] + ", {} state variables, {} samples".format(self.n_states, self.n_samples) + comma + uncertainty+ ">")
    
    def _copy(self, model):
        # create a clone of the class StochasticModel
        cls = self.__class__
        result = cls.__new__(cls)
        # copy the internal Gurobi model
        result._model = model.copy()
        for attribute, value in self.__dict__.items():
            if attribute == "_model":
                pass
            else:
                # copy all attributes that have not been assigned a value
                setattr(result, attribute, None)
                dict = {'value': value, 'target': result, 'attribute': attribute}
                # copy all uncertainties
                if attribute.startswith("uncertainty"):
                    setattr(result, attribute, {})
                    if attribute.startswith("uncertainty_rhs"):
                        deepcopy._copy_uncertainty_rhs(**dict)
                    elif attribute.startswith("uncertainty_coef"):
                        deepcopy._copy_uncertainty_coef(**dict)
                    elif attribute.startswith("uncertainty_obj"):
                        deepcopy._copy_uncertainty_obj(**dict)
                    elif attribute.startswith("uncertainty_mix"):
                        deepcopy._copy_uncertainty_mix(**dict)
                    else:
                        raise Exception("alien uncertainties added!")
                # copy all variables
                elif attribute in ["states", "local_copies", "alpha"]:
                    deepcopy._copy_vars(**dict)
                # copy all constraints
                elif attribute in ["cuts", "link_constrs"]:
                    deepcopy._copy_constrs(**dict)
                # copy probability measure
                elif attribute == "probability":
                    result.probability = None if value is None else list(value)
                # copy other numerical stuff
                else:
                    setattr(result, attribute, value)
        return result

    def _check_uncertainty(self, uncertainty, flag_dict, list_dim):
        # Make sure the input uncertainty is in the correct form. Return a
        # copied uncertainty to avoid making changes to mutable object given by the users.

    # In discrete case:
        # Uncertainty added by addConstr must be a dictionary (flag_dict=1, list_dim=1)
        # Uncertainty added by addVar must be a array-like (flag_dict=0, list_dim=1)
        # Uncertainty added by addConstrs and addVars must be a multidimensional array-like (flag_dict=0, list_dim>1)
    # In continuous case:
        # Uncertainty added by addConstr must be a dictionary (flag_dict=1, list_dim=1)
        # Uncertainty added by addVar must be a callable that generates a single number (flag_dict=0, list_dim=1)
        # Uncertainty added by addConstrs and addVars must be a callable that generates an array-like (flag_dict=0, list_dim>1)

        # All callable should take numpy RandomState as its only argument.
        if isinstance(uncertainty, abc.Mapping):
            uncertainty = dict(uncertainty)
            for key, item in uncertainty.items():
                if callable(item):
                    if not self._type:
                        # add uncertainty for the first time
                        self._type = "continuous"
                    else:
                        # already added uncertainty
                        if self._type != "continuous":
                            raise SampleSizeError(
                                self._model.modelName,
                                self.n_samples,
                                uncertainty,
                                "infinite"
                            )
                    try:
                        item(numpy.random)
                    except TypeError:
                        raise DistributionError(arg=False)
                    try:
                        float(item(numpy.random))
                    except (ValueError,TypeError):
                        raise DistributionError(ret=False)
                else:
                    try:
                        item = numpy.array(item, dtype='float64')
                    except ValueError:
                        raise ValueError("Scenarios must only contains numbers!")
                    if item.ndim != 1:
                        raise ValueError(
                            "dimension of the distribution is {} while \
                            dimension of the added object is {}!"
                            .format(item.ndim, 1)
                        )
                    uncertainty[key] = list(item)

                    if not self._type:
                        # add uncertainty for the first time
                        self._type = "discrete"
                        self.n_samples = len(item)
                    else:
                        # already added uncertainty
                        if self._type != "discrete":
                            raise SampleSizeError(
                                self._model.modelName,
                                "infinite",
                                {key:item},
                                len(item)
                            )
                        if self.n_samples != len(item):
                            raise SampleSizeError(
                                self._model.modelName,
                                self.n_samples,
                                {key:item},
                                len(item)
                            )
            if flag_dict == 0:
                raise TypeError("wrong uncertainty format!")
        elif isinstance(uncertainty, abc.Callable):
            try:
                sample = uncertainty(numpy.random)
            except TypeError:
                raise DistributionError(arg=False)
            if list_dim == 1:
                try:
                    float(sample)
                except (ValueError,TypeError):
                    raise DistributionError(ret=False)
            else:
                try:
                    sample = [float(item) for item in sample]
                except (ValueError,TypeError):
                    raise DistributionError(ret=False)
                if list_dim != len(uncertainty(numpy.random)):
                    raise ValueError(
                        "dimension of the distribution is {} while \
                        dimension of the added object is {}!"
                        .format(len(uncertainty(numpy.random)), list_dim)
                    )
            if not self._type:
                # add uncertainty for the first time
                self._type = "continuous"
            else:
                # already added uncertainty
                if self._type != "continuous":
                    raise SampleSizeError(
                        self._model.modelName,
                        self.n_samples,
                        uncertainty,
                        "infinite"
                    )
        elif isinstance(uncertainty, (abc.Sequence, numpy.ndarray)):
            uncertainty = numpy.array(uncertainty)
            if list_dim == 1:
                if uncertainty.ndim != 1:
                    raise ValueError("dimension of the scenarios is {} while \
                                     dimension of the added object is 1!"
                        .format(uncertainty.ndim)
                    )
                try:
                    uncertainty = [float(item) for item in uncertainty]
                except ValueError:
                    raise ValueError("Scenarios must only contains numbers!")
            else:
                # list to list
                if uncertainty.ndim != 2 or uncertainty.shape[1] != list_dim:
                    dim = None if uncertainty.ndim == 1 else uncertainty.shape[1]
                    raise ValueError("dimension of the scenarios is {} while \
                                     dimension of the added object is 1!"
                        .format(dim, uncertainty.ndim)
                    )
                try:
                    uncertainty = numpy.array(uncertainty, dtype='float64')
                except ValueError:
                    raise ValueError("Scenarios must only contains numbers!")
                uncertainty = [list(item) for item in uncertainty]
            if not self._type:
                self._type = "discrete"
                self.n_samples = len(uncertainty)
            else:
                if self._type != "discrete":
                    raise SampleSizeError(
                        self._model.modelName,
                        "infinite",
                        uncertainty,
                        len(uncertainty)
                    )
                if self.n_samples != len(uncertainty):
                    raise SampleSizeError(
                        self._model.modelName,
                        self.n_samples,
                        uncertainty,
                        len(uncertainty)
                    )
        else:
            raise TypeError("wrong uncertainty format!")

        return uncertainty


